In [25]:
import os
import sys
from collections import defaultdict, Counter
import time
from matplotlib import pyplot

import importlib
import abxportinf
import abxportinf._grouper
import abxportinf._optimize
import abxportinf._vectorizer
import abxportinf.busdef
import abxportinf.main 
import abxportinf.util
from abxportinf.busdef import BusDef
from abxportinf._optimize import MatchCost

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
block_root = '/ddr'
comp_json_path = os.path.join(block_root, 'ddr.json5')
ports = abxportinf.get_ports_from_json5(comp_json_path)

Warning ('regHADDR', 'PAR_REG_ADDR_WIDTH + 2') not correctly parsed
Warning ('phy_reg_addr', '-(PAR_REG_ADDR_WIDTH)') not correctly parsed


In [27]:
bus_spec_rootdir = '/bus-defs/specs'
                                
print('loading bus defs from specs')
bus_defs = []                
for root, dirs, fnames in os.walk(bus_spec_rootdir):
    #print((root, dirs, files))  
    for fname in fnames:             
        spec_path = os.path.join(root, fname)
        if BusDef.is_spec_bus_def(spec_path):     
            print('  - loading ', spec_path)
            bus_defs.extend(
                BusDef.bus_defs_from_spec(spec_path)
            )                
print('  - done')                                            
print('loaded {} bus definitions from specs'.format(len(bus_defs)))
print('  - total req ports', sum([bd.num_req_ports for bd in bus_defs]))
print('  - total opt ports', sum([bd.num_opt_ports for bd in bus_defs]))

loading bus defs from specs
  - loading  /bus-defs/specs/amba.com/AMBA2/AHB/r3p0_1/AHB_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA5/AHB5Target/r0p0_0/AHB5Target_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA5/AHB5Initiator/r0p0_0/AHB5Initiator_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/AXI4Stream/r0p0_1/AXI4Stream_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/AXI4/r0p0_0/AXI4_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/AXI4/r0p0_0/AXI4_RO_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/ACE-Lite/r0p0_0/ACE-Lite_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/APB4/r0p0_0/APB4_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/ATB/r0p0_0/ATB_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/ACP/r0p0_0/ACP_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/ACE/r0p0_0/ACE_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA3/AHBLite/r2p0_0/AHBLite_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA3/APB/r2p0_0/APB_rtl.json5


In [28]:
#stime = time.time()
pg_bus_mappings = abxportinf.get_bus_matches(ports, list(bus_defs))
#etime = time.time()
#print('total time: {}s'.format(etime-stime))

In [30]:
print(len(ports))
for lcost, port_group, bus_mappings in pg_bus_mappings:
    if lcost.dc != 0:
        continue
    best_mapping = bus_mappings[0]
    abxportinf.debug_bus_mapping(port_group, best_mapping)

296
bus_def{
	bus_type:{'vendor': 'amba.com', 'library': 'AMBA4', 'name': 'ACP', 'version': 'r0p0_0'},
	abstract_type:{'vendor': 'amba.com', 'library': 'AMBA4', 'name': 'ACP_rtl', 'version': 'r0p0_0'},
	driver_type:slave,
	num_req:22,
	num_opt:31,
}
  - fcost:17(n:0;w:17;d:0), cost:217(n:196;w:21;d:0)
  - mapped
    - cost:4(n:4;w:False;d:False), ('axi1_WREADY', 1, -1):('WREADY', 1, -1) req
    - cost:4(n:4;w:False;d:False), ('axi1_WVALID', 1, 1):('WVALID', 1, 1) req
    - cost:4(n:3;w:True;d:False), ('axi1_RID', 12, -1):('RID', None, -1) req
    - cost:4(n:4;w:False;d:False), ('axi1_BVALID', 1, -1):('BVALID', 1, -1) req
    - cost:4(n:4;w:False;d:False), ('axi1_ARREADY', 1, -1):('ARREADY', 1, -1) req
    - cost:4(n:4;w:False;d:False), ('axi1_ARVALID', 1, 1):('ARVALID', 1, 1) req
    - cost:4(n:4;w:False;d:False), ('axi1_RLAST', 1, -1):('RLAST', 1, -1) req
    - cost:4(n:4;w:False;d:False), ('axi1_BREADY', 1, 1):('BREADY', 1, 1) req
    - cost:4(n:4;w:False;d:False), ('axi1_AWVALID', 1

In [29]:
import numpy as np                       
import operator                         
import cvxopt                            
from collections import Counter                                               
from cvxopt import matrix, solvers       
from cvxopt.modeling import variable, dot, op                            
from cvxopt.modeling import sum as cvx_sum           
import editdistance as ed                                             
from abxportinf import util                                                          

solvers.options['show_progress'] = False

def map_ports_to_bus(ports, bus_def):
    """                           
    optimally map ports to bus definition {req, opt} ports by formulating
    as a convex LP and solving            
    """                            
    # get cost functions from closure, which takes into account specifics of
    # bus_def             
    match_cost, mapping_cost = get_cost_funcs(ports, bus_def)
                                           
    ports1 = list(ports)          
    ports2 = list(bus_def.req_ports)
    ports2.extend(bus_def.opt_ports)    

    m, n  = len(ports1), len(ports2)                               
    C = np.zeros((m, n))                                           
    for i, p1 in enumerate(ports1):                                      
        for j, p2 in enumerate(ports2):                    
            C[i,j] = match_cost(p1, p2).value
                                                          
    # swap phy ports with bus def so that the columns are always the ones
    # underdetermined                            
    swap = False                                                 
    if m > n:                                     
        swap = True                 
        m, n = n, m                                         
        ports1, ports2 = ports2, ports1                        
        C = C.T                                                                
                                                                               
    print('num variables', m*n)
    c = matrix(C.reshape(m*n))                  
    x = variable(m*n)                              
    constraints = [                   
        x >= 0,           
        x <= 1,                                                        
    ]           
    for i in range(m):                 
        #print('setting constraint', i*n, i*n+n)
        constraints.append(     
            cvx_sum(x[i*n:i*n+n]) == 1
        )                                
                                                
    # add constraints so max number of assignments to each port in ports2
    # is 1 as well                
    for j in range(n):
        #print(list(range(j, m*n, n)))
        constraints.append(        
            cvx_sum([x[jj] for jj in range(j, m*n, n)]) <= 1
        )                 
    op(                                                                  
        dot(c, x),                 
        constraints,               
    ).solve(solver='glpk')
    #).solve(solver=None)
    X = np.array(x.value).reshape(m,n) > 0.01
                                                             
    mapping = {ports1[i] : ports2[j] for i, j in np.argwhere(X)}
    if swap:                      
        mapping = {v:k for k, v in mapping.items()}
    cost = mapping_cost(mapping, ports, bus_def)
    return cost, mapping, match_cost
                                                   
def get_cost_funcs(ports, bus_def):          
    """                                                   
    determine cost functions in a closure with access to bus_def         
    """                                          
                                                                 
    def match_cost_func(phy_port, bus_port):      
                                    
        def name_dist(w1, w2):                              
            return ed.eval(w1, w2)                                               
            #return ed.eval(w1, w2) / max(len(w1), len(w2))                    
                                                                               
        # FIXME for now hardcode in functions that are used to get words from name
        # for both ports and buses                 
        p_words = util.words_from_name(phy_port[0].lower())
        b_words = bus_def.words_from_name(bus_port[0].lower())
        cost_n = 0                                                     
        for b_word in b_words:                              
            cost_n += min(map(lambda w: name_dist(b_word, w), p_words))
                                                
        return MatchCost(       
            # name attr mismatch      
            cost_n,                                                               
            # width mismatch                    
            (phy_port[1] != bus_port[1]),                                
            # direction mismatch        
            (phy_port[2] != bus_port[2]),
        )                               
                                         
    # NOTE this function closure actually includes the match_cost_func defined
    # above                              
    def mapping_cost_func(mapping, ports, bus_def):                      
        print('here?')
        umap_ports = set(ports) - set(mapping.keys())
        umap_busports = set(bus_def.req_ports) - set(mapping.values())
        cost = MatchCost.zero()                                             
        cost += sum([match_cost_func(p1, p2) for p1, p2 in mapping.items()])
        # penalize only width+direction for unmapped ports   
        cost += MatchCost(0,1,1)*len(umap_ports)                
        cost += MatchCost(0,1,1)*len(umap_busports)
        return cost                                
                                                
    return match_cost_func, mapping_cost_func
                                                

In [30]:
%load_ext line_profiler

print('port group size', len(dport_group))
print('bus def num req', dbus_def.num_req_ports)
print('bus def num opt', dbus_def.num_opt_ports)

stime = time.time()
map_ports_to_bus(dport_group, dbus_def)
etime = time.time()
print('total time:', etime-stime)
#%lprun -f map_ports_to_bus map_ports_to_bus(dport_group, dbus_def)

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
port group size 39
bus def num req 22
bus def num opt 46
num variables 2652
here?
total time: 1.9289271831512451
